In [1]:
import tensorflow as tf
import numpy as np

# TensorFlow的基本概念

## 张量,计算图和会话

TF中主要有如下三个部分：  
+ 张量——数据模型  
张量也就是多维数组，用于表示数据
+ 计算图——计算模型  
计算图是整个计算过程的表示，图中的节点表示运算操作，边表示运算操作之间的依赖关系
+ 会话——运行模型  
计算图中定义的每个运算操作必须在会话中执行

TensorFlow计算图表示如下含义：  
+ 每一个节点表示一个计算，或者说操作(operation)
+ 节点之间的边描述了计算之间的依赖关系。

<img src="images/f1.png" width="40%" align="left">

上述的图定义的是如下的运算：  
$add = a + b$   
对应的TF实现如下

In [14]:
a = tf.constant(1)
b = tf.constant(2)
add = tf.add(a,b)
print(a)
print(add)

Tensor("Const_12:0", shape=(), dtype=int32)
Tensor("Add_8:0", shape=(), dtype=int32)


上述定义的add是表示加法的这个操作，实际上，a和b也是表示操作——对应于常量赋值这个操作，这就是每个节点是一个计算的含义。 

节点a和b经过相加后得到节点add，这就是一个依赖关系。

因为是定义的计算节点，所以打印出的结果并不是实际的值。**要想打印出计算节点对应的值，必须要开启一个会话，在会话中进行图的计算**

In [17]:
# 使用tf.Session()创建一个会话
with tf.Session() as sess:
    # 使用 session.run()方法实际执行图中的计算节点，传入的参数是某个节点，返回的是该计算节点对应的值
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(add))

1
2
3


+ 以下是一个矩阵乘法的例子

In [4]:
# 定义两个常量赋值操作
a = tf.constant([ [3,3] ])
b = tf.constant([ [2], [3] ])

# 定义一个矩阵乘法操作
prod = tf.matmul(a, b)
print(prod)

Tensor("MatMul:0", shape=(1, 1), dtype=int32)


In [16]:
# 使用会话来运行上述定义的运算
with tf.Session() as sess:
    # 调用sess.run()方法来执行上述定义的运算
    result = sess.run(prod)
    print(result.__class__,"\n",result)

<class 'numpy.ndarray'> 
 [[15]]


## 变量的使用

变量有如下作用:  
1. 保存神经网络的参数
2. 获取神经网络中间的结果

In [13]:
# 定义一个变量并初始化为0的操作，
# 注意，这里只是定义初始化的方法（是用常量初始化还是用随机数初始化），但是并没有执行初始化
state = tf.Variable(0, name="state")
# 定义一个运算，使var每次加1
new_value = tf.add(state, 1)
# 定义赋值操作
update = tf.assign(state, new_value)
# 定义变量初始化操作
init = tf.global_variables_initializer()

# 创建会话运行上述定义的操作
with tf.Session() as sess:
    # 执行变量的初始化操作
    sess.run(init)
    print(sess.run(state))
    for _ in range(5):
        sess.run(update)
        print(sess.run(state))

0
1
2
3
4
5


## placeholder机制，Fetch和Feed

TensorFlow 提供了 placeholder 机制用于提供输入数据。placeholder 相当于定义了一个位置，这个位置中的数据在程序运行时再指定。

**placeholder通常用于传入一个batch的训练数据**。

Fetch是指一个session中运行多个operation。 
Feed是指以字典的形式传入多个数据。  

上述两个概念都是`Session.run()`方法的参数，可以查阅该函数的帮助文档

In [56]:
w1 = tf.Variable(tf.random_normal([2,3]))

# 定义一个placeholder需要提供类型（之后不可改变），shape不是必须的，但是提供了shape可以避免歧义和错误
x = tf.placeholder(tf.float32, shape=(3,1), name='x')

# 使用placeholder代替常量进行矩阵乘法操作的定义
y = tf.matmul(w1, x)

# 变量初始化操作
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(w1))
    # 执行y定义的操作的时候，需要传入placeholder x对应的值
    print(sess.run(y, feed_dict={x:[[1],[2],[3]]}))

[[-1.601076   -0.45815668 -0.31606627]
 [ 0.6450111  -0.37396577 -0.03890442]]
[[-3.465588  ]
 [-0.21963367]]


## 损失函数和优化器

## 简单使用案例

In [3]:
# 构造100个数据点
x_data = np.random.rand(100)
y_data = x_data * 0.1 + 0.2

#构造一个带训练的线性模型
a = tf.Variable(0.)
b = tf.Variable(0.)
y = a * x_data + b

# 二次代价函数
loss = tf.reduce_mean(tf.square(y - y_data))

# 定义优化器
optimizer = tf.train.GradientDescentOptimizer(0.2)

#最小化代价函数
train = optimizer.minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

# 构建session
with tf.Session() as sess:
    sess.run(init)
    for step in range(201):
        sess.run(train)
        if step%20 == 0:
            print(step, sess.run([a, b]))

0 [0.04929078, 0.098659955]
20 [0.099961706, 0.20001897]
40 [0.09997764, 0.20001116]
60 [0.099986926, 0.20000653]
80 [0.09999237, 0.20000382]
100 [0.09999553, 0.20000224]
120 [0.09999739, 0.20000131]
140 [0.09999846, 0.20000076]
160 [0.099999085, 0.20000045]
180 [0.099999465, 0.20000027]
200 [0.09999967, 0.20000017]


In [19]:
# 获取默认的计算图
tf.get_default_graph()

## 完整的神经网络程序示例

有三个步骤：  
1. 定义神经网络的结构和前向传播的输出结果。
2. 定义损失函数以及选择反向传播优化的算法 。
3. 生成会话（ tf.Session ）并且在训练、数据上反复运 行反向传播优化算法

作为示例的神经网络如下图所示：  
**它只有一个隐藏层，并且隐藏层没有使用激活函数。**

<img src="images/f2.png" width="70%" align="left">

In [63]:
# 使用numpy随机模拟数据
from numpy.random import RandomState

rnd = RandomState(29)
datasize = 128
X = rnd.rand(datasize,2)
Y = [ [int(x1+x2<1)] for (x1,x2) in X]

print(X[:5,:])
print(Y)

[[0.86375999 0.28490597]
 [0.07325639 0.7632372 ]
 [0.45271906 0.54229687]
 [0.72663578 0.84890511]
 [0.76819998 0.73314372]]
[[0], [1], [1], [0], [0], [1], [1], [1], [1], [0], [1], [0], [0], [0], [0], [1], [0], [1], [0], [1], [0], [0], [0], [0], [0], [1], [1], [1], [0], [0], [1], [0], [1], [0], [1], [1], [0], [1], [1], [1], [0], [0], [0], [0], [1], [0], [1], [1], [1], [1], [0], [0], [0], [1], [0], [1], [1], [1], [0], [0], [0], [1], [1], [0], [0], [1], [0], [1], [0], [0], [1], [1], [0], [0], [0], [1], [0], [1], [1], [0], [1], [0], [0], [1], [1], [0], [1], [1], [0], [0], [0], [1], [1], [0], [1], [1], [0], [1], [1], [1], [0], [1], [0], [0], [1], [1], [0], [1], [0], [0], [0], [1], [1], [1], [1], [1], [0], [0], [0], [0], [1], [1], [1], [0], [1], [0], [0], [0]]


In [69]:
#---------步骤1--------------
# 定义训练数据batch大小
batch_size = 8

# 定义神经网络的参数
# w1是输入层到隐藏层的权重矩阵, 2 x 3
w1 = tf.Variable(tf.random_normal([2,3]))
# w2是隐藏层到输出层的权重大小，3 x 1
w2 = tf.Variable(tf.random_normal([3,1]))

#定义placeholder，用于接受数据
# shape里第一个维度留None，可以用于动态适应输入的batch大小
x_batch = tf.placeholder(tf.float32, shape=(None, 2), name="x_batch")
y_batch = tf.placeholder(tf.float32, shape=(None, 1), name="y_batch")


#---------步骤2--------------
#定义神经网络的前向传播过程，这里采用矩阵运算
a = tf.matmul(x_batch, w1)
y = tf.matmul(a, w2)  # 这个 y 是神经网络输出的 y

# 定义损失函数和反向传播的优化算法
y = tf.sigmoid(y)
# 使用交叉熵作为损失函数
cross_entropy = - tf.reduce_mean(
    y_batch * tf.log(tf.clip_by_value(y, 1e-10, 10)) +
    (1-y_batch) * tf.log(tf.clip_by_value(y,1e-10, 10))
)
# 优化算法
train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cross_entropy)

#---------步骤3--------------
with tf.Session() as sess:
    # 初始化变量
    init = tf.global_variables_initializer()
    sess.run(init)
    # 打印训练之前权重矩阵的值
    print("w1")
    print(sess.run(w1))
    print("w2")
    print(sess.run(w2))
    
    # 设定训练轮数
    STEPS = 5000
    for i in range(STEPS):
        # 每次选择batch_size个样本进行运算，这里生成的是样本的index范围
        batch_start = (i * batch_size) % datasize
        batch_end = min(batch_start+batch_size, datasize)
        
        # 对每个batch进行训练并更新参数
        sess.run(train_step, feed_dict={x_batch:X[batch_start:batch_end,:], y_batch:Y[batch_start:batch_end]})
        
        # 每隔一段时间就打印一次训练过程中全部数据集上的交叉熵
        if i%500 == 0:
            total_cross_entropy = sess.run(cross_entropy , feed_dict={x_batch:X, y_batch:Y})
            print("经过 {} 次训练，所有数据上的交叉熵损失为：{:g}".format(i, total_cross_entropy))
            
    # 打印训练之后权重矩阵的值
    print("w1")
    print(sess.run(w1))
    print("w2")
    print(sess.run(w2))
        

w1
[[ 0.06869038  0.49156898  0.11386791]
 [ 0.8700317  -0.44391954  0.4978401 ]]
w2
[[-1.084994  ]
 [ 0.19721599]
 [-0.9770192 ]]
经过 0 次训练，所有数据上的交叉熵损失为：1.19841
经过 500 次训练，所有数据上的交叉熵损失为：0.502654
经过 1000 次训练，所有数据上的交叉熵损失为：0.195012
经过 1500 次训练，所有数据上的交叉熵损失为：0.0943532
经过 2000 次训练，所有数据上的交叉熵损失为：0.0556156
经过 2500 次训练，所有数据上的交叉熵损失为：0.0371082
经过 3000 次训练，所有数据上的交叉熵损失为：0.0267609
经过 3500 次训练，所有数据上的交叉熵损失为：0.0202841
经过 4000 次训练，所有数据上的交叉熵损失为：0.0158909
经过 4500 次训练，所有数据上的交叉熵损失为：0.0127333
w1
[[-0.86306465  2.570241   -1.2016271 ]
 [-0.22396241  1.8466288  -1.067801  ]]
w2
[[-1.0842199]
 [ 2.594463 ]
 [-2.4313126]]


+ 深层的神经网络模型

# MNIST手写数据集识别练习

In [20]:
from tensorflow.examples.tutorials.mnist import input_data

In [24]:
mnist = input_data.read_data_sets('./datasets/MNIST_data/', one_hot=True)

Extracting ./datasets/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./datasets/MNIST_data/t10k-labels-idx1-ubyte.gz


In [33]:
print(mnist.train.num_examples)
print(mnist.validation.num_examples)
print(mnist.test.num_examples)

55000
5000
10000


In [53]:
# 每一张图片是 28*28 像素，被拉成了一个 28*28=784的行向量
print("train.shape",mnist.train.images.shape)

# 每张图片对应的分类是一个10维向量，对应于10个分类
print("train_labels.shape",mnist.train.labels.shape)

# print(mnist.train.images[0])
print(mnist.train.labels[0])

train.shape (55000, 784)
train_labels.shape (55000, 10)
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [ ]:
mnist.train.next_batch()